###1. Load packages & pipeline

In [ ]:
!pip3 install konlpy

In [ ]:
!pip install stanza

In [ ]:
!pip install mecab-ko

In [66]:
import re
import pandas as pd
import stanza
from konlpy.tag import Okt
#from konlpy.tag import Mecab (Mecab was not able to load in Mac, so used the mecab_ko pacakge instead)
import mecab_ko as Mecab
from konlpy.tag import Kkma
from collections import Counter
import statistics

nlp = stanza.Pipeline(lang='ko', processors='tokenize')

2022-12-08 19:41:01 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2022-12-08 19:41:02 INFO: Loading these models for language: ko (Korean):
| Processor | Package |
-----------------------
| tokenize  | kaist   |

2022-12-08 19:41:02 INFO: Use device: cpu
2022-12-08 19:41:02 INFO: Loading: tokenize
2022-12-08 19:41:02 INFO: Done loading processors!


In [67]:
os.getcwd()

'c:\\Users\\sycho\\Personal_Data\\Git_repos\\klega_lexdiv\\notebook'

###2. Load the conll-U formated text file

In [68]:
# Load the conll-U formated text file

f = open('../data/kld_accuracy_small.txt', 'r', encoding='utf-16') # encoding='utf-16' 
## text file = 100 randomly sampled sentences from NIKL Korean learner corpus
### format: sent_id / text (=raw sentence) / hand tokenized based on each tokenizing rule for Stanza - Okt - Mecab - Kkma

f_lines = f.readlines() #read by lines
f.close()

len(f_lines)
f_lines[:10] #sample_test_lines

['# sent_id = 1\t\t\t\n',
 '# text = 나는 20년, 30년 후에 "내 인생은 대성공이다"고 할 수 있는 삶을 살고 싶습니다.\n',
 '1\t나는\t나+는\t나+는\t나+는\n',
 '2\t20년\t20년\t20+년\t20+년\n',
 '3\t,\t,\t,\t,\n',
 '4\t30년\t30년\t30+년\t30+년\n',
 '5\t후에\t후+에\t후+에\t후+에\n',
 '6\t"\t"\t"\t"\n',
 '7\t내\t내\t내\t나+의\n',
 '8\t인생은\t인생+은\t인생+은\t인생+은\n']

In [69]:
# Put the tokens into the list (pre-process)

tok_index = []
stanza = []
okt = []
mecab = []
kkma = []
text = []


for i, line in enumerate(f_lines):
    line = line.strip() # delete '\n'
    splits = line.split('\t') # split by '\t'

    if splits[0].startswith("# text"):
        text.append(splits[0][9:])
    if len(splits) > 2: # filter '\n\'
        try:
            idx = int(splits[0]) # filter 'data starting with '#'
            tok_index.append(idx)
            stanza.append(splits[1])
            okt.append(splits[2])
            mecab.append(splits[3])
            kkma.append(splits[4])

        except:
            print(i, tok_index[-5:], stanza[-5:], okt[-5:], mecab[-5:], kkma[-5:])
            print(line) #check if there is an error in idx
            # or use print(set(index))

In [70]:
# Check index and tokens

print(tok_index[:20])
print(stanza[:20])
print(okt[:20])
print(mecab[:20])
print(kkma[:20])

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 1, 2]
['나는', '20년', ',', '30년', '후에', '"', '내', '인생은', '대성공이다', '"', '고', '할', '수', '있는', '삶을', '살고', '싶습니다', '.', '그래서', ',']
['나+는', '20년', ',', '30년', '후+에', '"', '내', '인생+은', '대성공+이다', '"', '고', '할', '수', '있는', '삶+을', '살고', '싶습니다', '.', '그래서', ',']
['나+는', '20+년', ',', '30+년', '후+에', '"', '내', '인생+은', '대성공+이+다', '"', '고', '할', '수', '있+는', '삶+을', '살+고', '싶+습니다', '.', '그래서', ',']
['나+는', '20+년', ',', '30+년', '후+에', '"', '나+의', '인생+은', '대성공+이+다', '"', '고', '하+ㄹ', '수', '있+는', '삶+을', '살+고', '싶+습니다', '.', '그래서', ',']


In [71]:
# Put the tokens into the list (Stanza) --- This process is repeated four times

input_data_stanza = [] 

temp_stanza = [] 

i = 0
token_index = 0

while (i < len(tok_index)):
  if (int(tok_index[i])) == token_index + 1:
    temp_stanza.append(stanza[i])
    token_index += 1
    i = i +1

  else:	
    if temp_stanza:
      pos = [x.split("+") for x in temp_stanza]
      #pos = flatten(pos)
      input_data_stanza.append(pos)
    else:
      i = i +1
    token_index = 0 #new sentence, make the number of sentence to default number
    temp_stanza = []

  if i == (len(tok_index)-1):
    pos = [x.split("+") for x in temp_stanza]
    #pos = flatten(pos)
    input_data_stanza.append(pos)
    break

In [72]:
# Write a function to flatten the nested list 

def flatten(lst):
  result = []
  for item in lst:
    if type(item) == list:
      result += flatten(item)
    else:
      result += [item]
  return result

In [73]:
# Put the tokens into the list (Okt)

input_data_okt = [] 

temp_okt = [] 

i = 0
token_index = 0

while (i < len(tok_index)):
  if (int(tok_index[i])) == token_index + 1:
    temp_okt.append(okt[i])
    token_index += 1
    i = i +1

  else:	
    if temp_okt:
      pos = [x.split("+") for x in temp_okt]
      pos = flatten(pos)
      input_data_okt.append(pos)
    else:
      i = i +1
    token_index = 0 #new sentence, make the number of sentence to default number
    temp_okt = []

  if i == (len(tok_index)-1):
    pos = [x.split("+") for x in temp_okt]
    pos = flatten(pos)
    input_data_okt.append(pos)
    break
   

In [74]:
# Put the tokens into the list (Mecab)

input_data_mecab = [] #total test sentences

temp_mecab = [] #iterate through sentence

i = 0
token_index = 0

while (i < len(tok_index)):
  if (int(tok_index[i])) == token_index + 1:
    temp_mecab.append(mecab[i])
    token_index+= 1
    i = i +1

  else:	
    if temp_mecab:
      pos = [x.split("+") for x in temp_mecab]
      pos = flatten(pos)
      input_data_mecab.append(pos) 
    else:
      i = i +1
    
    token_index = 0 
    temp_mecab = []

  if i == (len(tok_index)-1):
    if temp_mecab:
      pos = [x.split("+") for x in temp_mecab]
      pos = flatten(pos)
      input_data_mecab.append(pos)
    break

In [75]:
# Put the tokens into the list (Kkma)

input_data_kkma = [] #total test sentences

temp_kkma = [] #iterate through sentence

i = 0
token_index = 0

while (i < len(tok_index)):
  if (int(tok_index[i])) == token_index + 1:
      temp_kkma.append(kkma[i])
      token_index+= 1
      i = i +1

  else:	
    if temp_kkma:
      pos = [x.split("+") for x in temp_kkma]
      pos = flatten(pos)
      input_data_kkma.append(pos) 
    else:
      i = i +1
    
    token_index = 0 
    temp_kkma = []

  if i == (len(tok_index)-1):
    pos = [x.split("+") for x in temp_kkma]
    pos = flatten(pos)
    input_data_kkma.append(pos)
    break

In [76]:
# Combine into a dafaframe

assert len(input_data_kkma) == len(input_data_mecab) == len(input_data_okt) == len(input_data_stanza) # assert that each bucket (kkma, mecab,.. ) processed the same amount of sentences
sent_id = list(range(1, len(input_data_kkma)+1)) # sent_id was missing in the original code. this value is valid when the sentence id starts with 1 with an increment of 1.

data_df = pd.concat([pd.Series(sent_id, name="sent_id"), pd.Series(text, name="text")], axis=1)

data_df = pd.concat([pd.Series(sent_id, name="sent_id"), pd.Series(text, name="text"), pd.Series(input_data_stanza, name="stanza"), 
                     pd.Series(input_data_okt, name="okt"), pd.Series(input_data_mecab, name="mecab"), pd.Series(input_data_kkma, name="kkma")], axis=1)

In [77]:
data_df.head()

,sent_id,text,stanza,okt,mecab,kkma
0,1,"나는 20년, 30년 후에 ""내 인생은 대성공이다""고 할 수 있는 삶을 살고 싶습니다.","[[나는], [20년], [,], [30년], [후에], [""], [내], [인생은...","[나, 는, 20년, ,, 30년, 후, 에, "", 내, 인생, 은, 대성공, 이다...","[나, 는, 20, 년, ,, 30, 년, 후, 에, "", 내, 인생, 은, 대성공...","[나, 는, 20, 년, ,, 30, 년, 후, 에, "", 나, 의, 인생, 은, ..."
1,2,"그래서, 계획을 세워하면 효율적인 시간에 하고 싶은 일 다 할 수 있다.","[[그래서], [,], [계획을], [세워하면], [효율적인], [시간에], [하고...","[그래서, ,, 계획, 을, 세워하면, 효율, 적, 인, 시간, 에, 하고, 싶은,...","[그래서, ,, 계획, 을, 세워, 하, 면, 효율, 적, 인, 시간, 에, 하, ...","[그래서, ,, 계획, 을, 세우, 어, 하면, 효율, 적, 인, 시간, 에, 하,..."
2,3,사고 후에 얼굴을 무섭게 생기게 된 사람들이 인생이 망하지 않고 고칠 수 있다면 좋겠다.,"[[사고], [후에], [얼굴을], [무섭게], [생기게], [된], [사람들이],...","[사고, 후, 에, 얼굴, 을, 무섭게, 생기게, 된, 사람, 들, 이, 인생, 이...","[사고, 후, 에, 얼굴, 을, 무섭, 게, 생기, 게, 된, 사람, 들, 이, 인...","[사고, 후, 에, 얼굴, 을, 무섭, 게, 생기, 게, 되, ㄴ, 사람, 들, 이..."
3,4,동물을 복제하는 것이 삼관없지만 인간 복제하면 여러 가지 영향도 마친다.,"[[동물을], [복제하는], [것이], [삼관없지만], [인간], [복제하면], [...","[동물, 을, 복제, 하는, 것, 이, 삼관, 없지만, 인간, 복제, 하면, 여러,...","[동물, 을, 복제, 하, 는, 것, 이, 삼관, 없, 지만, 인간, 복제, 하면,...","[동물, 을, 복제, 하, 는, 것, 이, 삼관, 없, 지만, 인간, 복제, 하면,..."
4,5,여성이 사회 진출하면 사회에서도 인정을 받고 사회적인 지위도 높아진다.,"[[여성이], [사회], [진출하면], [사회에서도], [인정을], [받고], [사...","[여성, 이, 사회, 진출, 하면, 사회, 에서, 도, 인정, 을, 받고, 사회, ...","[여성, 이, 사회, 진출, 하면, 사회, 에서, 도, 인정, 을, 받, 고, 사회...","[여성, 이, 사회, 진출, 하면, 사회, 에서, 도, 인정, 을, 받, 고, 사회..."


###3. Prediction with each tokenizer

In [83]:
# Put the automatically tokenized tokens into the list (Stanza) --- This process is repeated four times

stanza_tok = []

for sent in data_df['text']:
  doc = nlp(sent)
  for i, sentence in enumerate(doc.sentences):
      output = ([f'{token.text}' for token in sentence.tokens])
      stanza_tok.append(output)

In [84]:
# Put the automatically tokenized tokens into the list (Okt)

tagger = Okt()
okt_tok = []

for sent in data_df['text']:
  res = tagger.pos(str(sent))
  token = [x[0] for x in res]
  okt_tok.append(token)

In [85]:
# Put the automatically tokenized tokens into the list (Mecab)

tagger = Mecab.Tagger()
mecab_tok = []

def tagger_mm (sent):
	temp = tagger.parse(sent).split('\n')
	l1 = []
	l2 = []
	for t in temp:
		t = t.strip('"')
		t1 = t.split(',')[0]
		l1.append(t1)
	for l in l1:
		ls = l.split('\t')
		ls = ls[0]
		if str(ls) != "EOS":
			if str(ls) != '':
				l2.append(ls)
	return(l2)
  
for sent in data_df['text']:
  res = tagger_mm(str(sent))
  #print(res)
  mecab_tok.append(res)

In [86]:
# Put the automatically tokenized tokens into the list (Kkma)

tagger = Kkma()
kkma_tok = []

for sent in data_df['text']:
  res = tagger.pos(str(sent))
  token = [x[0] for x in res]
  kkma_tok.append(token)

In [87]:
# Combine into the previous dataframe

data_df = pd.concat([data_df, pd.Series(stanza_tok, name="stanza_tok"),
                     pd.Series(okt_tok, name="okt_tok"),
                     pd.Series(mecab_tok, name="mecab_tok"),
                     pd.Series(kkma_tok, name="kkma_tok")], axis=1)

data_df

,sent_id,text,stanza,okt,mecab,kkma,stanza_tok,okt_tok,mecab_tok,kkma_tok
0,1,"나는 20년, 30년 후에 ""내 인생은 대성공이다""고 할 수 있는 삶을 살고 싶습니다.","[[나는], [20년], [,], [30년], [후에], [""], [내], [인생은...","[나, 는, 20년, ,, 30년, 후, 에, "", 내, 인생, 은, 대성공, 이다...","[나, 는, 20, 년, ,, 30, 년, 후, 에, "", 내, 인생, 은, 대성공...","[나, 는, 20, 년, ,, 30, 년, 후, 에, "", 나, 의, 인생, 은, ...","[나는, 20년, ,, 30년, 후에, "", 내, 인생은, 대성공이다, "", 고, ...","[나, 는, 20년, ,, 30년, 후, 에, "", 내, 인생, 은, 대성공, 이다...","[나, 는, 20, 년, 30, 년, 후, 에, 내, 인생, 은, 대성공, 이, 다...","[나, 는, 20, 년, ,, 30, 년, 후, 에, "", 내, 인생, 은, 대성공..."
1,2,"그래서, 계획을 세워하면 효율적인 시간에 하고 싶은 일 다 할 수 있다.","[[그래서], [,], [계획을], [세워하면], [효율적인], [시간에], [하고...","[그래서, ,, 계획, 을, 세워하면, 효율, 적, 인, 시간, 에, 하고, 싶은,...","[그래서, ,, 계획, 을, 세워, 하, 면, 효율, 적, 인, 시간, 에, 하, ...","[그래서, ,, 계획, 을, 세우, 어, 하면, 효율, 적, 인, 시간, 에, 하,...","[그래서, ,, 계획을, 세워하면, 효율적인, 시간에, 하고, 싶은, 일, 다, 할...","[그래서, ,, 계획, 을, 세워하면, 효율, 적, 인, 시간, 에, 하고, 싶은,...","[그래서, 계획, 을, 세워, 하, 면, 효율, 적, 인, 시간, 에, 하, 고, ...","[그리하, 여서, ,, 계획, 을, 세우, 어, 하면, 효율적, 이, ㄴ, 시간, ..."
2,3,사고 후에 얼굴을 무섭게 생기게 된 사람들이 인생이 망하지 않고 고칠 수 있다면 좋겠다.,"[[사고], [후에], [얼굴을], [무섭게], [생기게], [된], [사람들이],...","[사고, 후, 에, 얼굴, 을, 무섭게, 생기게, 된, 사람, 들, 이, 인생, 이...","[사고, 후, 에, 얼굴, 을, 무섭, 게, 생기, 게, 된, 사람, 들, 이, 인...","[사고, 후, 에, 얼굴, 을, 무섭, 게, 생기, 게, 되, ㄴ, 사람, 들, 이...","[사고, 후에, 얼굴을, 무섭게, 생기게, 된, 사람들이, 인생이, 망하지, 않고,...","[사고, 후, 에, 얼굴, 을, 무섭게, 생기, 게, 된, 사람, 들, 이, 인생,...","[사고, 후, 에, 얼굴, 을, 무섭, 게, 생기, 게, 된, 사람, 들, 이, 인...","[사고, 후, 에, 얼굴, 을, 무섭, 게, 생기, 게, 되, ㄴ, 사람, 들, 이..."
3,4,동물을 복제하는 것이 삼관없지만 인간 복제하면 여러 가지 영향도 마친다.,"[[동물을], [복제하는], [것이], [삼관없지만], [인간], [복제하면], [...","[동물, 을, 복제, 하는, 것, 이, 삼관, 없지만, 인간, 복제, 하면, 여러,...","[동물, 을, 복제, 하, 는, 것, 이, 삼관, 없, 지만, 인간, 복제, 하면,...","[동물, 을, 복제, 하, 는, 것, 이, 삼관, 없, 지만, 인간, 복제, 하면,...","[동물을, 복제하는, 것이, 삼관없지만, 인간, 복제하면, 여러, 가지, 영향도, ...","[동물, 을, 복제, 하는, 것, 이, 삼, 관, 없지만, 인간, 복제, 하면, 여...","[동물, 을, 복제, 하, 는, 것, 이, 삼관, 없, 지만, 인간, 복제, 하, ...","[동물, 을, 복제, 하, 는, 것, 이, 삼관, 없, 지만, 인간, 복제, 하, ..."
4,5,여성이 사회 진출하면 사회에서도 인정을 받고 사회적인 지위도 높아진다.,"[[여성이], [사회], [진출하면], [사회에서도], [인정을], [받고], [사...","[여성, 이, 사회, 진출, 하면, 사회, 에서, 도, 인정, 을, 받고, 사회, ...","[여성, 이, 사회, 진출, 하면, 사회, 에서, 도, 인정, 을, 받, 고, 사회...","[여성, 이, 사회, 진출, 하면, 사회, 에서, 도, 인정, 을, 받, 고, 사회...","[여성이, 사회, 진출하면, 사회에서도, 인정을, 받고, 사회적인, 지위도, 높아진...","[여성, 이, 사회, 진출, 하면, 사회, 에서도, 인정, 을, 받고, 사회, 적,...","[여성, 이, 사회, 진출, 하, 면, 사회, 에서, 도, 인정, 을, 받, 고, ...","[여성, 이, 사회, 진출, 하, 면, 사회, 에서, 도, 인정, 을, 받, 고, ..."


###4. Calculate accuracy (precision - recall - F1)

In [93]:
# tmp
## Mecab
num_total_gold_token_mecab = [len(x) for x in input_data_mecab]

num_intersection_pos_mecab = []
for index, li in enumerate(input_data_mecab):
    num_intersection_pos_mecab.append(len(list((Counter(input_data_mecab[index]) & Counter(mecab_tok[index])).elements())))
    
# calculate the average accuracy across all sentences
mecab_p = [i / j for i, j in zip(num_intersection_pos_mecab, num_total_gold_token_mecab)]

In [88]:
# precision

## stanza
num_total_gold_token_stanza = [len(x) for x in input_data_stanza]

num_intersection_pos_stanza = []
for index, li in enumerate(input_data_stanza):
    num_intersection_pos_stanza.append(len(list((Counter(input_data_stanza[index]) & Counter(stanza_tok[index])).elements())))
    

# calculate the average accuracy across all sentences
stanza_p = [i / j for i, j in zip(num_intersection_pos_stanza, num_total_gold_token_stanza)]

# merge into the data_df
data_df = pd.concat([data_df, pd.Series(stanza_p, name='stanza_p')], axis=1)


## okt
num_total_gold_token_okt = [len(x) for x in input_data_okt]

num_intersection_pos_okt = []
for index, li in enumerate(input_data_okt):
    num_intersection_pos_okt.append(len(list((Counter(input_data_okt[index]) & Counter(okt_tok[index])).elements())))
    

# calculate the average accuracy across all sentences
okt_p = [i / j for i, j in zip(num_intersection_pos_okt, num_total_gold_token_okt)]

# merge into the data_df
data_df = pd.concat([data_df, pd.Series(okt_p, name='okt_p')], axis=1)


## Mecab
num_total_gold_token_mecab = [len(x) for x in input_data_mecab]

num_intersection_pos_mecab = []
for index, li in enumerate(input_data_mecab):
    num_intersection_pos_mecab.append(len(list((Counter(input_data_mecab[index]) & Counter(mecab_tok[index])).elements())))
    

# calculate the average accuracy across all sentences
mecab_p = [i / j for i, j in zip(num_intersection_pos_mecab, num_total_gold_token_mecab)]

# merge into the data_df
data_df = pd.concat([data_df, pd.Series(mecab_p, name='mecab_p')], axis=1)


## Kkma
num_total_gold_token_kkma = [len(x) for x in input_data_kkma]

num_intersection_pos_kkma = []
for index, li in enumerate(input_data_kkma):
    num_intersection_pos_kkma.append(len(list((Counter(input_data_kkma[index]) & Counter(kkma_tok[index])).elements())))
    

# calculate the average accuracy across all sentences
kkma_p = [i / j for i, j in zip(num_intersection_pos_kkma, num_total_gold_token_kkma)]

# merge into the data_df
data_df = pd.concat([data_df, pd.Series(kkma_p, name='kkma_p')], axis=1)


print("Stanza precision:", statistics.mean(stanza_p))
print("Okt precision:", statistics.mean(okt_p))
print("Mecab precision:", statistics.mean(mecab_p))
print("Kkma precision:", statistics.mean(kkma_p))

TypeError: unhashable type: 'list'

In [ ]:
# Recall

## stanza
num_total_predict_stanza = [len(x) for x in stanza_tok]

num_intersection_pos_stanza = []
for index, li in enumerate(input_data_stanza):
    num_intersection_pos_stanza.append(len(list((Counter(input_data_stanza[index]) & Counter(stanza_tok[index])).elements())))
    

# calculate the average accuracy across all sentences
stanza_r = [i / j for i, j in zip(num_intersection_pos_stanza, num_total_predict_stanza)]

# merge into the data_df
data_df = pd.concat([data_df, pd.Series(stanza_r, name='stanza_r')], axis=1)


## okt
num_total_predict_okt = [len(x) for x in okt_tok]

num_intersection_pos_okt = []
for index, li in enumerate(input_data_okt):
    num_intersection_pos_okt.append(len(list((Counter(input_data_okt[index]) & Counter(okt_tok[index])).elements())))
    

# calculate the average accuracy across all sentences
okt_r = [i / j for i, j in zip(num_intersection_pos_okt, num_total_predict_okt)]

# merge into the data_df
data_df = pd.concat([data_df, pd.Series(okt_r, name='okt_r')], axis=1)


## Mecab
num_total_predict_mecab = [len(x) for x in mecab_tok]

num_intersection_pos_mecab = []
for index, li in enumerate(input_data_mecab):
    num_intersection_pos_mecab.append(len(list((Counter(input_data_mecab[index]) & Counter(mecab_tok[index])).elements())))
    

# calculate the average accuracy across all sentences
mecab_r = [i / j for i, j in zip(num_intersection_pos_mecab, num_total_predict_mecab)]

# merge into the data_df
data_df = pd.concat([data_df, pd.Series(mecab_r, name='mecab_r')], axis=1)


## Kkma
num_total_predict_kkma = [len(x) for x in kkma_tok]

num_intersection_pos_kkma = []
for index, li in enumerate(input_data_kkma):
    num_intersection_pos_kkma.append(len(list((Counter(input_data_kkma[index]) & Counter(kkma_tok[index])).elements())))
    

# calculate the average accuracy across all sentences
kkma_r = [i / j for i, j in zip(num_intersection_pos_kkma, num_total_predict_kkma)]

# merge into the data_df
data_df = pd.concat([data_df, pd.Series(kkma_r, name='kkma_r')], axis=1)


print("Stanza recall:", statistics.mean(stanza_r))
print("Okt recall:", statistics.mean(okt_r))
print("Mecab recall:", statistics.mean(mecab_r))
print("Kkma recall:", statistics.mean(kkma_r))

Stanza recall: 0.9588670905312272
Okt recall: 0.8999636446646948
Mecab recall: 0.9291278611942471
Kkma recall: 0.8937333502704202


In [ ]:
data_df

,sent_id,text,stanza,okt,mecab,kkma,stanza_tok,okt_tok,mecab_tok,kkma_tok,stanza_p,okt_p,mecab_p,kkma_p,stanza_r,okt_r,mecab_r,kkma_r
0,1,그것은 그 연예인 팬 분들이 꼭 가고 싶다라고 해서 소용이 없는 제품을 많이 사서 ...,"[그것은, 그, 연예인, 팬, 분들이, 꼭, 가고, 싶다라고, 해서, 소용이, 없는...","[그것, 은, 그, 연예인, 팬, 분, 들, 이, 꼭, 가고, 싶다라고, 해서, 소...","[그것, 은, 그, 연예인, 팬, 분, 들, 이, 꼭, 가, 고, 싶, 다, 라고,...","[그것, 은, 그, 연예인, 팬, 분, 들, 이, 꼭, 가, 고, 싶, 다, 라고,...","[그것은, 그, 연예인, 팬, 분들이, 꼭, 가고, 싶다라고, 해서, 소용이, 없는...","[그것, 은, 그, 연예인, 팬, 분, 들, 이, 꼭, 가고, 싶다라고, 해서, 소...","[그것, 은, 그, 연예인, 팬, 분, 들, 이, 꼭, 가, 고, 싶, 다, 라고,...","[그것, 은, 그, 연예인, 팬, 분, 들, 이, 꼭, 가, 고, 싶, 다, 라고,...",1.0,0.961538,1.000,0.941176,1.0000,0.925926,1.000000,0.969697
1,2,학부모들의 아이들의 성적이 수준별 이동 수업을 통해 향상시킨다는 답변이 많다.,"[학부모들의, 아이들의, 성적이, 수준별, 이동, 수업을, 통해, 향상시킨다는, 답...","[학부모, 들, 의, 아이, 들, 의, 성적, 이, 수준별, 이동, 수업, 을, 통...","[학부모, 들, 의, 아이, 들, 의, 성적, 이, 수준, 별, 이동, 수업, 을,...","[학부모, 들, 의, 아이, 들, 의, 성적, 이, 수준, 별, 이동, 수업, 을,...","[학부모들의, 아이들의, 성적이, 수준별, 이동, 수업을, 통해, 향상시킨다는, 답...","[학부모, 들, 의, 아이, 들, 의, 성적, 이, 수준별, 이동, 수업, 을, 통...","[학부모, 들, 의, 아이, 들, 의, 성적, 이, 수준, 별, 이동, 수업, 을,...","[학부모, 들, 의, 아이, 들, 의, 성적, 이, 수준, 별, 이동, 수업, 을,...",1.0,1.000000,1.000,1.000000,1.0000,1.000000,1.000000,1.000000
2,3,요즘에는 이런 가족의 모습도 보기 어렵게 되었습니다.,"[요즘에는, 이런, 가족의, 모습도, 보기, 어렵게, 되었습니다, .]","[요즘, 에는, 이런, 가족, 의, 모습, 도, 보기, 어렵게, 되었습니다, .]","[요즘, 에, 는, 이런, 가족, 의, 모습, 도, 보, 기, 어렵, 게, 되, 었...","[요즘, 에, 는, 이런, 가족, 의, 모습, 도, 보, 기, 어렵, 게, 되, 었...","[요즘에는, 이런, 가족의, 모습도, 보기, 어렵게, 되었습니다, .]","[요즘, 에는, 이런, 가족, 의, 모습, 도, 보기, 어렵게, 되었습니다, .]","[요즘, 에, 는, 이런, 가족, 의, 모습, 도, 보, 기, 어렵, 게, 되, 었...","[요즘, 에, 는, 이런, 가족, 의, 모습, 도, 보, 기, 어렵, 게, 되, 었...",1.0,1.000000,1.000,1.000000,1.0000,1.000000,1.000000,1.000000
3,4,그리고 그런 사람들은 긴혐도 많아서 회사에 도움이 크다.,"[그리고, 그런, 사람들은, 긴혐도, 많아서, 회사에, 도움이, 크다, .]","[그리고, 그런, 사람, 들, 은, 긴혐, 도, 많아서, 회사, 에, 도움, 이, ...","[그리고, 그런, 사람, 들, 은, 긴혐, 도, 많, 아서, 회사, 에, 도움, 이...","[그리고, 그런, 사람, 들, 은, 긴혐, 도, 많, 아서, 회사, 에, 도움, 이...","[그리고, 그런, 사람들은, 긴혐도, 많아서, 회사에, 도움이, 크다, .]","[그리고, 그런, 사람, 들, 은, 긴혐, 도, 많아서, 회사, 에, 도움, 이, ...","[그리고, 그런, 사람, 들, 은, 긴, 혐도, 많, 아서, 회사, 에, 도움, 이...","[그리, 고, 그러, ㄴ, 사람, 들, 은, 길, ㄴ, 혀, ㅁ, 도, 많, 아서,...",1.0,1.000000,0.875,0.812500,1.0000,1.000000,0.875000,0.619048
4,5,전통 예술을 지키려는 사람도 많아질 것이다.,"[전통, 예술을, 지키려는, 사람도, 많아질, 것이다, .]","[전통, 예술, 을, 지키려는, 사람, 도, 많아질, 것, 이다, .]","[전통, 예술, 을, 지키, 려는, 사람, 도, 많, 아, 질, 것, 이, 다, .]","[전통, 예술, 을, 지키, 려는, 사람, 도, 많, 아, 지, ㄹ, 것, 이, 다...","[전통, 예술을, 지키려는, 사람도, 많아질, 것이다, .]","[전통, 예술, 을, 지키려는, 사람, 도, 많아질, 것, 이다, .]","[전통, 예술, 을, 지키, 려는, 사람, 도, 많, 아, 질, 것, 이, 다, .]","[전통, 예술, 을, 지키, 려는, 사람, 도, 많, 아, 지, ㄹ, 것, 이, 다...",1.0,1.000000,1.000,1.000000,1.0000,1.000000,1.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,먼저 인터넷은 여러 나라 사람들이 사용할 수 있으므로 의견도 다양하다.,"[먼저, 인터넷은, 여러, 나라, 사람들이, 사용할, 수, 있으므로, 의견도, 다양...","[먼저, 인터넷, 은, 여러, 나라, 사람, 들, 이, 사용할, 수, 있으므로, 의...","[먼저, 인터넷, 은, 여러, 나라, 사람, 들, 이, 사용, 할, 수, 있, 으므...","[먼저, 인터넷, 은, 여러, 나라, 사람, 들, 이, 사용, 하, ㄹ, 수, 있,...","[먼저, 인터넷은, 여러, 나라, 사람들이, 사용할, 수, 있으므로, 의견도, 다양...","[먼저, 인터넷, 은, 여러, 나라, 사람, 들, 이, 사용, 할, 수, 있으므로,...","[먼저, 인터넷, 은, 여러, 나라, 사람, 들, 이, 사용, 할, 수, 있, 으므...","[먼저, 인터넷, 은, 여러, 나라, 사람, 들, 이, 사용, 하, ㄹ, 수, 있,...",1.0,0.812500,1.000,1.000000,1.0000,0.812500,1.000000,1.000000
96,97,미신을 믿은 사람들의 의견도 한번 듣고 싶다.,"[미신을, 믿은, 사람들의, 의견도, 한번, 듣고, 싶다, .]","[미신, 을, 믿은, 사람, 들, 의, 의견, 도, 한번, 듣고, 싶다, .]","[미신, 을, 믿, 은, 사람, 들, 의, 의견, 도, 한, 번, 듣, 고, 싶, ...","[미신, 을, 믿, 은, 사람, 들, 의, 의견, 도, 한, 번, 듣, 고, 싶, ...","[미신을, 믿은, 사람들의, 의견도, 한번, 듣고, 싶다, .]","[미신, 을, 믿은, 사람, 들, 의, 의견, 도, 한번, 듣고, 싶다, .]","[미신, 을, 믿, 은, 사람, 들, 의, 의견, 도, 한, 번, 듣, 고, 싶, ...","[미신, 을, 믿, 은, 사람, 들, 의, 의견, 도, 한번, 듣, 고, 싶, 다, .]",1.0,1.000000,1.000,0.875000,1.0000,1.000000,1.000000,0.933333
97,98,미국 보기에는 한국인이 이렇게 다들 열심히 공부와 시험을 잘해서 우리가 한국처럼 하...,"[미국, 보기에는, 한국인이, 이렇게, 다들, 열심히, 공부와, 시험을, 잘해서, ...","[미국, 보기, 에는, 한국인, 이, 이렇게, 다, 들, 열심히, 공부, 와, 시험...","[미국, 보, 기, 에, 는, 한국인, 이, 이렇게, 다, 들, 열심히, 공부, 와...","[미국, 보, 기, 에, 는, 한국인, 이, 이렇게, 다, 들, 열심히, 공부, 와...","[미국, 보기에는, 한국인이, 이렇게, 다들, 열심히, 공부와, 시험

In [ ]:
# F1

stanza_f = 2 * ((data_df["stanza_p"] * data_df["stanza_r"])/(data_df["stanza_p"] + data_df["stanza_r"]))
okt_f = 2 * ((data_df["okt_p"] * data_df["okt_r"])/(data_df["okt_p"] + data_df["okt_r"]))
mecab_f = 2 * ((data_df["mecab_p"] * data_df["mecab_r"])/(data_df["mecab_p"] + data_df["mecab_r"]))
kkma_f = 2 * ((data_df["kkma_p"] * data_df["kkma_r"])/(data_df["kkma_p"] + data_df["kkma_r"]))

# merge into the data_df
data_df = pd.concat([data_df, pd.Series(stanza_f, name='stanza_f')], axis=1)
data_df = pd.concat([data_df, pd.Series(okt_f, name='okt_f')], axis=1)
data_df = pd.concat([data_df, pd.Series(mecab_f, name='mecab_f')], axis=1)
data_df = pd.concat([data_df, pd.Series(kkma_f, name='kkma_f')], axis=1)

print("Stanza f1:", statistics.mean(stanza_f))
print("Okt f1:", statistics.mean(okt_f))
print("Mecab f1:", statistics.mean(mecab_f))
print("Kkma f1:", statistics.mean(kkma_f))

Stanza f1: 0.9672513681360455
Okt f1: 0.9070603645197822
Mecab f1: 0.9359412640886796
Kkma f1: 0.9047505471716741


###5. Save result

In [ ]:
data_df

,sent_id,text,stanza,okt,mecab,kkma,stanza_tok,okt_tok,mecab_tok,kkma_tok,...,mecab_p,kkma_p,stanza_r,okt_r,mecab_r,kkma_r,stanza_f,okt_f,mecab_f,kkma_f
0,1,그것은 그 연예인 팬 분들이 꼭 가고 싶다라고 해서 소용이 없는 제품을 많이 사서 ...,"[그것은, 그, 연예인, 팬, 분들이, 꼭, 가고, 싶다라고, 해서, 소용이, 없는...","[그것, 은, 그, 연예인, 팬, 분, 들, 이, 꼭, 가고, 싶다라고, 해서, 소...","[그것, 은, 그, 연예인, 팬, 분, 들, 이, 꼭, 가, 고, 싶, 다, 라고,...","[그것, 은, 그, 연예인, 팬, 분, 들, 이, 꼭, 가, 고, 싶, 다, 라고,...","[그것은, 그, 연예인, 팬, 분들이, 꼭, 가고, 싶다라고, 해서, 소용이, 없는...","[그것, 은, 그, 연예인, 팬, 분, 들, 이, 꼭, 가고, 싶다라고, 해서, 소...","[그것, 은, 그, 연예인, 팬, 분, 들, 이, 꼭, 가, 고, 싶, 다, 라고,...","[그것, 은, 그, 연예인, 팬, 분, 들, 이, 꼭, 가, 고, 싶, 다, 라고,...",...,1.000,0.941176,1.0000,0.925926,1.000000,0.969697,1.000000,0.943396,1.000000,0.955224
1,2,학부모들의 아이들의 성적이 수준별 이동 수업을 통해 향상시킨다는 답변이 많다.,"[학부모들의, 아이들의, 성적이, 수준별, 이동, 수업을, 통해, 향상시킨다는, 답...","[학부모, 들, 의, 아이, 들, 의, 성적, 이, 수준별, 이동, 수업, 을, 통...","[학부모, 들, 의, 아이, 들, 의, 성적, 이, 수준, 별, 이동, 수업, 을,...","[학부모, 들, 의, 아이, 들, 의, 성적, 이, 수준, 별, 이동, 수업, 을,...","[학부모들의, 아이들의, 성적이, 수준별, 이동, 수업을, 통해, 향상시킨다는, 답...","[학부모, 들, 의, 아이, 들, 의, 성적, 이, 수준별, 이동, 수업, 을, 통...","[학부모, 들, 의, 아이, 들, 의, 성적, 이, 수준, 별, 이동, 수업, 을,...","[학부모, 들, 의, 아이, 들, 의, 성적, 이, 수준, 별, 이동, 수업, 을,...",...,1.000,1.000000,1.0000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2,3,요즘에는 이런 가족의 모습도 보기 어렵게 되었습니다.,"[요즘에는, 이런, 가족의, 모습도, 보기, 어렵게, 되었습니다, .]","[요즘, 에는, 이런, 가족, 의, 모습, 도, 보기, 어렵게, 되었습니다, .]","[요즘, 에, 는, 이런, 가족, 의, 모습, 도, 보, 기, 어렵, 게, 되, 었...","[요즘, 에, 는, 이런, 가족, 의, 모습, 도, 보, 기, 어렵, 게, 되, 었...","[요즘에는, 이런, 가족의, 모습도, 보기, 어렵게, 되었습니다, .]","[요즘, 에는, 이런, 가족, 의, 모습, 도, 보기, 어렵게, 되었습니다, .]","[요즘, 에, 는, 이런, 가족, 의, 모습, 도, 보, 기, 어렵, 게, 되, 었...","[요즘, 에, 는, 이런, 가족, 의, 모습, 도, 보, 기, 어렵, 게, 되, 었...",...,1.000,1.000000,1.0000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
3,4,그리고 그런 사람들은 긴혐도 많아서 회사에 도움이 크다.,"[그리고, 그런, 사람들은, 긴혐도, 많아서, 회사에, 도움이, 크다, .]","[그리고, 그런, 사람, 들, 은, 긴혐, 도, 많아서, 회사, 에, 도움, 이, ...","[그리고, 그런, 사람, 들, 은, 긴혐, 도, 많, 아서, 회사, 에, 도움, 이...","[그리고, 그런, 사람, 들, 은, 긴혐, 도, 많, 아서, 회사, 에, 도움, 이...","[그리고, 그런, 사람들은, 긴혐도, 많아서, 회사에, 도움이, 크다, .]","[그리고, 그런, 사람, 들, 은, 긴혐, 도, 많아서, 회사, 에, 도움, 이, ...","[그리고, 그런, 사람, 들, 은, 긴, 혐도, 많, 아서, 회사, 에, 도움, 이...","[그리, 고, 그러, ㄴ, 사람, 들, 은, 길, ㄴ, 혀, ㅁ, 도, 많, 아서,...",...,0.875,0.812500,1.0000,1.000000,0.875000,0.619048,1.000000,1.000000,0.875000,0.702703
4,5,전통 예술을 지키려는 사람도 많아질 것이다.,"[전통, 예술을, 지키려는, 사람도, 많아질, 것이다, .]","[전통, 예술, 을, 지키려는, 사람, 도, 많아질, 것, 이다, .]","[전통, 예술, 을, 지키, 려는, 사람, 도, 많, 아, 질, 것, 이, 다, .]","[전통, 예술, 을, 지키, 려는, 사람, 도, 많, 아, 지, ㄹ, 것, 이, 다...","[전통, 예술을, 지키려는, 사람도, 많아질, 것이다, .]","[전통, 예술, 을, 지키려는, 사람, 도, 많아질, 것, 이다, .]","[전통, 예술, 을, 지키, 려는, 사람, 도, 많, 아, 질, 것, 이, 다, .]","[전통, 예술, 을, 지키, 려는, 사람, 도, 많, 아, 지, ㄹ, 것, 이, 다...",...,1.000,1.000000,1.0000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,먼저 인터넷은 여러 나라 사람들이 사용할 수 있으므로 의견도 다양하다.,"[먼저, 인터넷은, 여러, 나라, 사람들이, 사용할, 수, 있으므로, 의견도, 다양...","[먼저, 인터넷, 은, 여러, 나라, 사람, 들, 이, 사용할, 수, 있으므로, 의...","[먼저, 인터넷, 은, 여러, 나라, 사람, 들, 이, 사용, 할, 수, 있, 으므...","[먼저, 인터넷, 은, 여러, 나라, 사람, 들, 이, 사용, 하, ㄹ, 수, 있,...","[먼저, 인터넷은, 여러, 나라, 사람들이, 사용할, 수, 있으므로, 의견도, 다양...","[먼저, 인터넷, 은, 여러, 나라, 사람, 들, 이, 사용, 할, 수, 있으므로,...","[먼저, 인터넷, 은, 여러, 나라, 사람, 들, 이, 사용, 할, 수, 있, 으므...","[먼저, 인터넷, 은, 여러, 나라, 사람, 들, 이, 사용, 하, ㄹ, 수, 있,...",...,1.000,1.000000,1.0000,0.812500,1.000000,1.000000,1.000000,0.812500,1.000000,1.000000
96,97,미신을 믿은 사람들의 의견도 한번 듣고 싶다.,"[미신을, 믿은, 사람들의, 의견도, 한번, 듣고, 싶다, .]","[미신, 을, 믿은, 사람, 들, 의, 의견, 도, 한번, 듣고, 싶다, .]","[미신, 을, 믿, 은, 사람, 들, 의, 의견, 도, 한, 번, 듣, 고, 싶, ...","[미신, 을, 믿, 은, 사람, 들, 의, 의견, 도, 한, 번, 듣, 고, 싶, ...","[미신을, 믿은, 사람들의, 의견도, 한번, 듣고, 싶다, .]","[미신, 을, 믿은, 사람, 들, 의, 의견, 도, 한번, 듣고, 싶다, .]","[미신, 을, 믿, 은, 사람, 들, 의, 의견, 도, 한, 번, 듣, 고, 싶, ...","[미신, 을, 믿, 은, 사람, 들, 의, 의견, 도, 한번, 듣, 고, 싶, 다, .]",...,1.000,0.875000,1.0000,1.000000,1.000000,0.933333,1.000000,1.000000,1.000000,0.903226
97,98,미국 보기에는 한국인이 이렇게 다들 열심히 공부와 시험을 잘해서 우리가 한국처럼 하...,"[미국, 보기에는, 한국인이, 이렇게, 다들, 

In [ ]:
data_df.to_csv("kld_tokenizer_f1.tsv", sep="\t", encoding='utf-16')